# Introduction to Vcc Glitch Attacks

In [1]:
%%bash
cd ../../hardware/victims/firmware/
mkdir -p glitch-simple-lab2 && cp -r glitch-simple/* $_

In [1]:
PLATFORM = "CW308_STM32F0"
CRYPTO_TARGET="NONE"

Change `glitch1()` to  `glitch_infinite()`.

In [2]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/glitch-simple-lab1
make PLATFORM=$1 CRYPTO_TARGET=$2

rm -f -- glitchsimple-CW308_STM32F0.hex
rm -f -- glitchsimple-CW308_STM32F0.eep
rm -f -- glitchsimple-CW308_STM32F0.cof
rm -f -- glitchsimple-CW308_STM32F0.elf
rm -f -- glitchsimple-CW308_STM32F0.map
rm -f -- glitchsimple-CW308_STM32F0.sym
rm -f -- glitchsimple-CW308_STM32F0.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- glitchsimple.s simpleserial.s stm32f0_hal.s stm32f0_hal_lowlevel.s
rm -f -- glitchsimple.d simpleserial.d stm32f0_hal.d stm32f0_hal_lowlevel.d
rm -f -- glitchsimple.i simpleserial.i stm32f0_hal.i stm32f0_hal_lowlevel.i
.
-------- begin --------
arm-none-eabi-gcc (GNU Tools for Arm Embedded Processors 7-2018-q2-update) 7.3.1 20180622 (release) [ARM/embedded-7-branch revision 261907]
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: glitchsimple.c
arm-none-eabi-gcc -c -mcpu=cortex-m0 -I. -mthumb -mf

## Attack Script

### Setup

Now that we've studied the code and have an objective, we can start looking at how to control the glitch module via Python. We'll start by connecting to and setting up the ChipWhisperer, then programming it. As usual, make sure you modify `fw_path` with the path to the file you built in the last step.

In [38]:
%run "Helper_Scripts/CWLite_Connect.ipynb"

In [39]:
%run "Helper_Scripts/Setup_Target_Generic.ipynb"

In [3]:
# uncomment based on your target
fw_path = "../../hardware/victims/firmware/glitch-simple-lab2/glitchsimple-CW308_STM32F0.hex"
%run "Helper_Scripts/Program_STM.ipynb"
#%run "Helper_Scripts/Program_STM.ipynb"
#%run "Helper_Scripts/No_Programmer.ipynb"

In [7]:
program_target(scope, fw_path)

Detected known STMF32: STM32F071xx/STM32F072xx
Extended erase (0x44), this can take ten seconds or more
Attempting to programming 4179 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 4179 bytes


Since the firmware enters an infinite loop, we'll need to reset the target between glitch attempts. Be sure to uncomment the reset pin for your target:

In [40]:
import time
def reset_target(scope):
    scope.io.nrst = 'low'
    #scope.io.pdic = 'low'
    time.sleep(0.05)
    scope.io.nrst = 'high'
    #scope.io.pdic = 'high'

Now let's make sure the firmware works as we expect. We should get `"hello\nA"` back after resetting the target.

In [41]:
scope.clock.clkgen_freq = 250000000
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "glitch_only"
scope.glitch.trigger_src = "ext_single"
scope.glitch.width = 30
scope.glitch.offset = 10.2
scope.glitch.repeat = 14
scope.io.glitch_lp = True
print(scope.glitch)
from collections import namedtuple
Range = namedtuple('Range', ['min', 'max', 'step'])

clk_src     = clkgen
width       = 30.078125
width_fine  = 0
offset      = 10.15625
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 0
repeat      = 14
output      = glitch_only



In [45]:
reset_target(scope)
target.ser.flush()
time.sleep(1)
num_char = target.ser.inWaiting()
resp = target.ser.read(num_char)
print(resp)

40000 200 200 0
40000 200 200 1
40000 200 200 2
40000 200 200 3
40000 200 200 4
40000 200 200 5
40000 200 200 6
40000 200 200 7


In [48]:
scope.glitch.ext_offset = 2114
print(scope.glitch)

clk_src     = clkgen
width       = 30.078125
width_fine  = 0
offset      = 10.15625
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 2114
repeat      = 14
output      = glitch_only



In [52]:
from tqdm import trange
for j in trange(100):
    target.ser.flush()

    scope.arm()

    while target.isDone() is False:
        timeout -= 1
        time.sleep(0.01)

    time.sleep(0.1)
    num_char = target.ser.inWaiting()
    resp = target.ser.read(num_char)
    nums = resp.split(" ")
    if int(nums[0]) != 40000:
        print(resp)

 25%|██▌       | 25/100 [00:02<00:08,  8.36it/s]

2147523648 200 200 1648



 26%|██▌       | 26/100 [00:03<00:08,  8.40it/s]

1090872452 200 200 1649



 30%|███       | 30/100 [00:03<00:08,  8.50it/s]

...

 54%|█████▍    | 54/100 [00:06<00:05,  8.50it/s]

3309406116 200 200 1678



 83%|████████▎ | 83/100 [00:09<00:02,  8.45it/s]

2164317860 200 200 1708



100%|██████████| 100/100 [00:11<00:00,  8.51it/s]


In [68]:
target.ser.flush()
scope.glitch.trigger_src = "ext_continuous"
for j in range(100):
    line = ""
    while "\n" not in line:
        time.sleep(0.1)
        num_char = target.ser.inWaiting()
        line += target.ser.read(num_char)
    
    nums = line.split(" ")
    if int(nums[0]) != 40000:
        print(line)
    #print(line, end = "")
    
scope.glitch.trigger_src = "ext_single"

39999 200 200 10862

2241764416 200 200 10863

3310853045 200 200 10865

3344473149 200 200 10883

1090880800 200 200 10884



In [ ]:
scope.dis()
target.dis()